GOAL: Build a Deep Learning Model (CNN) to predict whether handwritten images are the numbers 0, 1, 2,...,9

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

#### Data Import

In [ ]:
# import the image data into dataframes
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#### EDA (Exploratory Data Analysis)

In [ ]:
# some basic checks
x_train.shape

this tells us we have 60,000 images each of 28*28 pixels

In [ ]:
x_test.shape

what about the images themselves ?

In [ ]:
image_index = 3466 # any number between 0 and 59,999 (so we can choose from the TRAINING set)

plt.imshow(x_train[image_index], cmap = "Greys")

In [ ]:
# actual number classification
y_train[image_index]

In [ ]:
# Exercise - go ahead and check a few images in the TEST set

In [ ]:
image_index = 3466 # any number between 0 and 9,999 (so we can choose from the TRAINING set)

plt.imshow(x_test[image_index], cmap = "Greys")

In [ ]:
# actual number classification
y_test[image_index]

#### Data Wrangling

Data prep for Deep Learning requires a number of pre-processes:

1. reshaping the dataset
2. convert pixel values to float (decimals)
3. normalise (scale) the RGB colour codes

In [ ]:
# STEP 1 reshape the underlying data for the images (adds a dimension) so that we can work with the Keras API 
# this also effectively turns our input training data into a TENSOR
x_train_rs = x_train.reshape(x_train.shape[0],28,28,1) # adding a dimension

In [ ]:
# aside - what does x_train_rs look like ?
x_train_rs[image_index]

In [ ]:
# STEP 2 convert pixel values to float (decimals)
x_train_fl = x_train_rs.astype("float32")

In [ ]:
# STEP 3 normalise (scale) the RGB colour codes by dividing by the number of RGB 
# (red, green and blue) values (255)
x_train_nm = x_train_fl / 255

In [ ]:
# quick check
x_train_nm.shape

In [ ]:
# how does tensorflow convert a single image into a tensor format ??
x_train_nm[image_index]

In [ ]:
# Exercise -
# a) perform the same data wrangling steps on the TEST set
# b) display the image and the (normalised) tensor for TEST image 547

In [ ]:
# a) perform the same data wrangling steps on the TEST set

# STEP 1 reshape the underlying data for the images (adds a dimension) so that we can work with the Keras API 
# this also effectively turns our input testing data into a TENSOR
x_test_rs = x_test.reshape(x_test.shape[0],28,28,1) # adding a dimension
# aside - what does x_test_rs look like ?
x_test_rs[image_index]
# STEP 2 convert pixel values to float (decimals)
x_test_fl = x_test_rs.astype("float32")
# STEP 3 normalise (scale) the RGB colour codes by dividing by the number of RGB 
# (red, green and blue) values (255)
x_test_nm = x_test_fl / 255

In [ ]:
# b) display the image and the (normalised) tensor for TEST image 547
plt.imshow(x_test[547], cmap = "Greys")

In [ ]:
x_test_nm[547]

#### Building and Running the CNN

In [ ]:
# BUILD / define the model (STEP 1)
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D # types of hidden layers in CNN

myModel = models.Sequential() # sequential model setup
myModel.add(layers.Conv2D(28, kernel_size = (3,3), input_shape = (28,28,1))) # 1st convolutional hidden layer
myModel.add(layers.MaxPooling2D(pool_size = (2,2))) # pooling layer for "down-sampling" (for preventing overfitting)
myModel.add(Flatten()) # flattens tensor into array
myModel.add(layers.Dense(128, activation = tf.nn.relu)) # dense layer with RELU activation
myModel.add(layers.Dropout(0.2)) # helps with overfitting (we exclude some random data from the previous layer)
myModel.add(layers.Dense(10, activation = tf.nn.softmax)) # dense layer (LOGITS) with softmax activation

In [ ]:
# Exercise - see if you COMPILE the model (STEP 2)
# and FIT the model (i.e. train STEP 3) 

In [ ]:
# COMPILE the model (STEP 2)
# we use adam is an enhanced gradient descent algorithm
myModel.compile(optimizer="adam", 
              loss="sparse_categorical_crossentropy",
              metrics = "accuracy")

# FIT the model (i.e. train STEP 3)
# NB default batch_size is 32
myModel.fit(x_train_nm, y_train, epochs=10)

across 10 epochs (10 times thru the 60,000 training images), we have a loss of 0.02 (close to zero) and accuracy of 0.9933 (close to 1) - a good model on the TRAINING set.

We now need to check performance on the TEST set

In [ ]:
# SETP 4 Model EValuation (on the TEST set)
myModel.evaluate(x_test_nm, y_test)

313 passes on the TEST set (b/c we have 10,000 images and batch_size is 32)

TEST performance is also good: 0.06 loss and 0.98 accuracy

#### Performance Benchmarking

In [ ]:
# Exercise 

# a) see if you can PREDICT (STEP 5) using the trained model on image 878

In [ ]:
allPreds = myModel.predict(x_test_nm) # all predictions on the test set

In [ ]:
allPreds[878] # the prediction for test image 878

above is a PROBABILITY ARRAY which shows the probability that the image is the number 0, 1,...,9

In [ ]:
# easier way to get straight to the PREDICTED classification
allPreds[878].argmax() # argmax takes the index of the highest value from the probability array

In [ ]:
# Exercise (continued) -

# b) compare the prediction with the actual classification (y_test)
# c) isolate all the INCORRECT image predictions as a list

In [ ]:
# b) compare the prediction with the actual classification (y_test)
y_test[878]

so image no. 878 was CORRECTLY predicted as the number 8

In [ ]:
plt.imshow(x_test[878], cmap = "Greys") # view the actual image

In [ ]:
# c) isolate all the INCORRECT image predictions as a list

myErrs = [] # initialise a list to capture INCORRECT predictions

# loop over all the TEST images
for i in range(0,10000):
  # if statement which identifies INCORRECT predictions
  if y_test[i] != allPreds.argmax(axis = 1)[i]:
    myErrs.append(i)

myErrs # indices of incorrectly predicted images

In [ ]:
# how many did we get wrong ?
len(myErrs)

In [ ]:
# check one of the incorrect predictions
y_test[4123] # actual classifcation

In [ ]:
allPreds[4123].argmax()

In [ ]:
# how does the image look ?
plt.imshow(x_test[4123], cmap = "Greys")

finally lets take a look at a confusion matrix representation of test set performance...

In [ ]:
# Confusion Matrix

# function to plot the confusion matrix below

def plot_confusion_matrix(cm, 
                          classes, 
                          figname, 
                          normalize=False, 
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    
    # show the confusion matrix as RELATIVE or ABSOLUTE
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    plt.figure(figsize=(8,8)) # 8*8 inches
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)

    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd' # if showing a RELATIVE confusion matrix, then show values to 2 DPs
    thresh = cm.max() / 2.

    # display of the confusion matrix "squares"
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(figname)

In [ ]:
# Exercise -
# a) display the raw values in the confusion matrix
# tell me how many times we incorrectly predicted the number 4 (actual) for 9 (prediction)
# b) call the custom function to plot the confusion matrix


In [ ]:
from sklearn.metrics import confusion_matrix

cm =  confusion_matrix(y_test, allPreds.argmax(axis = 1))
cm

In [ ]:
# b) call the custom function to plot the confusion matrix

import numpy as np

plot_confusion_matrix(cm, 
                          list(range(0,9)), 
                          figname = "myCM.png", 
                          normalize=False, 
                          title='Confusion matrix for TEST set',
                          cmap=plt.cm.Blues)